<a href="https://colab.research.google.com/github/QingfangLiu/DS_learning/blob/main/GNN_learning_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)

In [ ]:
edge_index
# here 4 edges: 0 to 1, 1 to 0, 1 to 2, 2 to 1
# undirected

tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

In [ ]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [ ]:
x

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [ ]:
data = Data(x=x, edge_index=edge_index)

In [ ]:
data

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
data.validate(raise_on_error=True)

True

In [ ]:
data.keys()

['x', 'edge_index']

In [ ]:
data['x']

tensor([[-1.],
        [ 0.],
        [ 1.]])

In [ ]:
data.num_edge_features

0

In [ ]:
data.num_edge_types

1

In [ ]:
data.num_edges

4

In [ ]:
data.num_features

1

In [ ]:
data.num_node_features

1

In [ ]:
data.num_nodes

3

In [ ]:
from torch_geometric.datasets import Planetoid

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [ ]:
dataset

Cora()

In [ ]:
print('# of node features:',dataset[0].num_node_features)
print('# of nodes:',dataset[0].num_nodes)
print('# of edges:',dataset[0].num_edges)

# of node features: 1433
# of nodes: 2708
# of edges: 10556


In [ ]:
dataset[0].edge_index.shape

torch.Size([2, 10556])

In [ ]:
dataset.num_classes

7

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
# training for 200 epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()